### TODOS 6-4-2020
### 1. Get better generator data. Currently missing a lot of coal and nuclear
### 2. Figure out how to estimate lines reactance and apparent power limit
### 3. Map Generator, Loads, and Transmission Lines to LMP Pnode

In [12]:
import json
import os
from functools import partial
from time import sleep
import re

import pandas as pd
import pypsa
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import pyproj
from shapely.ops import transform


import geopandas as gpd
from geovoronoi import coords_to_points, points_to_coords, voronoi_regions_from_coords, calculate_polygon_areas
from geovoronoi.plotting import subplot_for_map, plot_voronoi_polys_with_points_in_area

# Load Lines

In [13]:
# load data
with open('data/pjm_system_map_export/backbone_lines.json') as f:
    data = json.load(f)
print("number of data points:", len(data["results"]))

# check data
for x in data["results"]:
    if x["layerId"] != 9:
        raise("Failed layerId check!")
    if x["layerName"] != "Backbone Transmission Lines":
        raise("Failed layerName check!")
    if x["geometryType"] != "esriGeometryPolyline":
        raise("Failed geometryType check!")
    if len(x["geometry"]["paths"]) != 1:
        raise("Failed paths length check!")
print("passed data check")



# # load data
# with open('data/pjm_system_map_export/non_backbone_lines_120_138_161_230_kv.json') as f:
#     data2 = json.load(f)
# print("number of data points:", len(data2["results"]))

# # check data
# for x in data2["results"]:
#     if x["layerId"] != 10:
#         raise("Failed layerId check!")
#     if x["layerName"] != "Transmission Lines":
#         raise("Failed layerName check!")
#     if x["geometryType"] != "esriGeometryPolyline":
#         raise("Failed geometryType check!")
#     if len(x["geometry"]["paths"]) != 1:
#         raise("Failed paths length check!")
# print("passed data check")

# # add to data
# data["results"] += data2["results"]


# # load data
# with open('data/pjm_system_map_export/non_backbone_lines_69_115_kv.json') as f:
#     data3 = json.load(f)
# print("number of data points:", len(data3["results"]))

# # check data
# for x in data3["results"]:
#     if x["layerId"] != 10:
#         raise("Failed layerId check!")
#     if x["layerName"] != "Transmission Lines":
#         raise("Failed layerName check!")
#     if x["geometryType"] != "esriGeometryPolyline":
#         raise("Failed geometryType check!")
#     if len(x["geometry"]["paths"]) != 1:
#         raise("Failed paths length check!")
# print("passed data check")


# # add to data
# data["results"] += data3["results"]


# construct geojson
geojson = {
    "type": "FeatureCollection",
    "crs": { "type": "name", "properties": { "name": "urn:ogc:def:crs:OGC:1.3:CRS84" } }, # this is not the appropriate projection
    "features": [
    {
        "type": "Feature",
        "properties" : d["attributes"],
        "geometry" : {
            "type": "LineString",
            "coordinates": d["geometry"]["paths"][0],
            }
     } for d in data["results"]]
}

print("finished constructing geojson")

# write geojson to disk
output = open("data/pjm_system_map_export/lines.geojson", 'w')
json.dump(geojson, output)
output.close()
print("geojson file has been written out to disk!")

# load geojson as a geodataframe
lines = gpd.read_file("data/pjm_system_map_export/lines.geojson")
print("geojson file has been read as a geodataframe!")

# replace NULL and None values with NaN
lines = lines.replace({"Null": np.nan, None: np.nan, "":np.nan})

number of data points: 732
passed data check
finished constructing geojson
geojson file has been written out to disk!
geojson file has been read as a geodataframe!


In [14]:
# lines.geometry.plot(figsize=(25, 10))

In [15]:
# lines.head()

In [16]:
# line_rating = pd.read_csv("data/oasis/line_rating.csv")

# node_list = pd.read_excel("data/lmp-bus-model.xlsx", skiprows=2)
# node_list.Voltage = node_list.Voltage.apply(lambda x: float(x.replace("KV", "")))
# node_list.columns = ["pnode_id", "zone", "substation", "voltage", "equipment", "type"]

In [17]:
# not_found = []

# for x in line_rating.substation.unique():
#     if x not in node_list.substation.to_list():
#         not_found.append(x)

# len(not_found)

In [18]:
# lines[lines["TO_LINE_NAME"] == "Null"]

# Load Substation

In [19]:
# load data
with open('data/pjm_system_map_export/substation.json') as f:
    data = json.load(f)
print("number of data points:", len(data["results"]))


# check data
for x in data["results"]:
    if x["layerId"] != 2:
        raise("Failed layerId check!")
    if x["layerName"] != "Substations":
        raise("Failed layerName check!")
    if x["geometryType"] != "esriGeometryPoint":
        raise("Failed geometryType check!")
print("passed data check")


# load data
with open('data/pjm_system_map_export/substation_non_member.json') as f:
    data2 = json.load(f)
print("number of data points:", len(data2["results"]))


# check data
for x in data2["results"]:
    if x["layerId"] != 26:
        raise("Failed layerId check!")
    if x["layerName"] != "Non-Member Substations":
        raise("Failed layerName check!")
    if x["geometryType"] != "esriGeometryPoint":
        raise("Failed geometryType check!")
print("passed data check")

# add to data
data["results"] += data2["results"]


# construct geojson
geojson = {
    "type": "FeatureCollection",
    "crs": { "type": "name", "properties": { "name": "urn:ogc:def:crs:OGC:1.3:CRS84" } }, # this is not the appropriate projection
    "features": [
    {
        "type": "Feature",
        "properties" : d["attributes"],
        "geometry" : {
            "type": "Point",
            "coordinates": [d["geometry"]["x"], d["geometry"]["y"]],
            }
     } for d in data["results"]]
}

print("finished constructing geojson")


# write geojson to disk
output = open("data/pjm_system_map_export/substation.geojson", 'w')
json.dump(geojson, output)
output.close()
print("geojson file has been written out to disk!")

# load geojson as a geodataframe
substations = gpd.read_file("data/pjm_system_map_export/substation.geojson")
print("geojson file has been read as a geodataframe!")

# replace NULL and None values with NaN
substations = substations.replace({"Null": np.nan, None: np.nan, "":np.nan})

number of data points: 6341
passed data check
number of data points: 6532
passed data check
finished constructing geojson
geojson file has been written out to disk!
geojson file has been read as a geodataframe!


In [20]:
# substations.geometry.plot(figsize=(15, 10))

In [21]:
# substations.head()

In [22]:
# check if any substations in lines can't be found in the substation list

for x in lines.SUBSTATION_A_GLOBALID.unique():
    if x not in substations.SUBSTATION_GLOBALID.unique():
        print(x)

print()

for x in lines.SUBSTATION_B_GLOBALID.unique():
    if x not in substations.SUBSTATION_GLOBALID.unique():
        print(x)

nan

nan
{C1B8B982-30DB-4CB0-8ABE-07AA5ED59FE9}
{5DA81B32-4CF2-4253-BEAD-4E60375DBB6C}
{14AD9BDD-513B-439D-9DEB-DBE59F014A88}


# Map Substations in System Map to PNodes

In [23]:
# load substation labels

with open('data/pjm_system_map_export/substation_labels.json') as f:
    data = json.load(f)
print("number of data points:", len(data["results"]))


# check data
for x in data["results"]:
    if x["layerId"] != 3:
        raise("Failed layerId check!")
    if x["geometryType"] != "esriGeometryPoint":
        raise("Failed geometryType check!")
print("passed data check")


# construct geojson
geojson = {
    "type": "FeatureCollection",
    "crs": { "type": "name", "properties": { "name": "urn:ogc:def:crs:OGC:1.3:CRS84" } }, # this is not the appropriate projection
    "features": [
    {
        "type": "Feature",
        "properties" : d["attributes"],
        "geometry" : {
            "type": "Point",
            "coordinates": [d["geometry"]["x"], d["geometry"]["y"]],
            }
     } for d in data["results"]]
}

print("finished constructing geojson")


# write geojson to disk
output = open("data/pjm_system_map_export/substation_labels.geojson", 'w')
json.dump(geojson, output)
output.close()
print("geojson file has been written out to disk!")

# load geojson as a geodataframe
substation_labels = gpd.read_file("data/pjm_system_map_export/substation_labels.geojson")
print("geojson file has been read as a geodataframe!")

# replace NULL and None values with NaN
substation_labels = substation_labels.replace({"Null": np.nan, None: np.nan, "":np.nan})

number of data points: 6343
passed data check
finished constructing geojson
geojson file has been written out to disk!
geojson file has been read as a geodataframe!


In [24]:
# zone by zone and voltage level by voltage level

# load pnode_list
node_list = pd.read_excel("data/lmp-bus-model.xlsx", skiprows=2)
node_list.Voltage = node_list.Voltage.apply(lambda x: float(x.replace("KV", "")))
node_list.columns = ["pnode_id", "zone", "substation", "voltage", "equipment", "type"]
node_list.substation = node_list.substation.astype(str)


# replace zonal name to accomodate matching with equiplist
zonalMap = {'AECO': "AEC",
            'AEP': "AEP",
            'APS': "APS",
            'ATSI': "ATSI",
            'BGE': "BGE",
            'COMED': "ComEd",
            'DAY': "Dayton",
            'DEOK': "DEOK",
            'DOM': "Dominion",
            'DPL': "DPL",
            'DUQ': "DL",
            'EKPC': "EKPC",
            'JCPL': "JCPL",
            'METED': "ME",
            'OVEC': "OVEC HQ",
            'PECO': "PECO",
            'PENELEC': "PENELEC",
            'PEPCO': "PEPCO",
            'PPL': "PPL",
            'PSEG': "PSEG",
            'RECO': "RE"}

node_list.zone = node_list.zone.replace(zonalMap)

In [25]:
# check that zone are one-one match between the two dataframes

for x in node_list.zone.unique():
    if x not in substation_labels.PLANNING_ZONE_NAME.to_list():
        print(x)

for x in substation_labels.PLANNING_ZONE_NAME.dropna().unique():
    if x not in node_list.zone.to_list():
        print(x)

In [27]:
# convert raw pjm zone json data exported from system map to geojson for geopandas ingesting

# load pjm zones
with open('data/pjm_system_map_export/pjm_zone.json') as f:
    zone = json.load(f)
    
print("number of zones:", len(zone["results"]))


# check data
for x in zone["results"]:
    if x["layerId"] != 17:
        raise("Failed layerId check!")
    if x["layerName"] != "PJM Zones":
        raise("Failed layerName check!")
    if x["geometryType"] != "esriGeometryPolygon":
        raise("Failed geometryType check!")


# construct geojson
geojson = {
    "type": "FeatureCollection",
    "crs": { "type": "name", "properties": { "name": "urn:ogc:def:crs:OGC:1.3:CRS84" } },
    "features": []
}


# add to geojson
for d in zone["results"]:
    tmp = {"type": "Feature", "properties" : d["attributes"], "geometry":{"type": None, "coordinates": None}}
    
    if len(d["geometry"]["rings"]) == 1 or d["value"] == "EKPC": #TODO: hardcoding EKPC for now, which is a donut shape
        tmp["geometry"]["type"] = "Polygon"
        geojson["features"].append(tmp)
        tmp["geometry"]["coordinates"] = d["geometry"]["rings"]
    else:
        tmp["geometry"]["type"] = "MultiPolygon"
        geojson["features"].append(tmp)
        tmp["geometry"]["coordinates"] = [[x] for x in d["geometry"]["rings"]]
    

# write geojson to disk
output = open("data/pjm_system_map_export/pjm_zone.geojson", 'w')
json.dump(geojson, output)
output.close()
print("geojson file has been written out to disk!")

# load geojson as a geodataframe
pjm_zone = gpd.read_file("data/pjm_system_map_export/pjm_zone.geojson")
print("geojson file has been read as a geodataframe!")

# replace NULL and None values with NaN
pjm_zone = pjm_zone.replace({"Null": np.nan, None: np.nan, "":np.nan})


number of zones: 21
geojson file has been written out to disk!
geojson file has been read as a geodataframe!


In [28]:
# matched substation to zone based on geoemtry

zoneGeometryMap = dict(zip(pjm_zone.PLANNING_ZONE_NAME, pjm_zone.geometry))

substation_labels["matched_zone"] = np.nan

for index, row in substation_labels.iterrows():
    if row["NAME"] is not np.nan:
        for zone, geometry in zoneGeometryMap.items():
            if row["geometry"].within(geometry):
                substation_labels.loc[index, "matched_zone"] = zone

In [ ]:
# set up weighted levenshtein score
from weighted_levenshtein import lev

insert_costs = np.full((128,), 1, dtype=np.float64)
delete_costs = np.full((128,), 5, dtype=np.float64)
substitute_costs = np.full((128, 128), 99999, dtype=np.float64)

def weighted_lev(word1, word2):
    return lev(word1, word2,
               insert_costs=insert_costs,
               delete_costs=delete_costs,
               substitute_costs=substitute_costs)


def find_match(word1, choices):
    tmp_word1 = word1.lower()
    tmp_choices = list(map(lambda x: x.lower(), choices))
    scores = list(map(lambda x: weighted_lev(tmp_word1, x), tmp_choices))
    min_index = scores.index(min(scores))
    return (choices[min_index], scores[min_index])


find_match("FELCTY", ['Taylor', 'Felicity'])

In [248]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# matching
# as more matches ahppen, gradually decrease in the threshold for fuzzy match
# the idea is to match with gradually decreasing confidence

# first iteration: perfect match confidence, zone matches or voltage matches
# second iteration: high match confidence, zone matches or voltage matches
# third iteration: high match confidence
# fourth iteration: intermediate match confidence
# fifth iteration: remaining substations

# revise substation_labels dataframe as needed
substation_labels["VOLTAGE"] = substation_labels.VOLTAGE.astype(float)
substation_labels = substation_labels.drop_duplicates()
substation_labels["NAME"] = substation_labels["NAME"].astype(str).apply(lambda x: re.sub(r'[^\w]', '', x))

# initialize dictionaries and lists
yet_to_be_matched_sublabels = substation_labels.SUBSTATION_GLOBALID.to_list()
yet_to_be_matched_nodes = node_list.groupby("substation")["zone"].unique().to_dict()
mapping = {}


# # get pnode and list of voltages using pnode_list and equiplist
# node_voltage_map = {}
# for x in node_list.substation.unique():
#     tmp_voltage = equiplist[equiplist.STATION == x].VOLTAGE.apply(lambda x: float(x.replace(" KV", ""))).unique()
#     node_voltage_map[x] = tmp_voltage


matching_iterations = {
    0: {"threshold": 100, "zone_check": True, "fuzz score": True, "description": "exact match in same zone"},
    1: {"threshold": 90, "zone_check": True, "fuzz score": True, "description": "high match in same zone"},
    2: {"threshold": 95, "zone_check": False, "fuzz score": True, "description": "high match in all zones"},
    3: {"threshold": 5, "zone_check": True, "fuzz score": False, "description": "medium match in same zone with weighted levenshtein"},
    4: {"threshold": 5, "zone_check": False, "fuzz score": False, "description": "medium match in same zone with weighted levenshtein"},
    5: {"threshold": 10, "zone_check": True, "fuzz score": False, "description": "low match in same zone with weighted levenshtein"},
    6: {"threshold": np.inf, "zone_check": False, "fuzz score": False, "description": "remaining same zone with weighted levenshtein"}
}


for index, threshold in enumerate([100, 90, 95, 5, 5, 10, np.inf]):

    for name in list(yet_to_be_matched_nodes):
        # TODO: need to add condition to skip substations that do not have >= 69 voltages

        zone = yet_to_be_matched_nodes[name]
        
        tmp_sub_labels = substation_labels[substation_labels.SUBSTATION_GLOBALID.isin(yet_to_be_matched_sublabels)]

        # TODO: tmp. adding zone check here
        if index in [0, 1, 3, 5]:
            tmp_sub_labels = tmp_sub_labels[(tmp_sub_labels.PLANNING_ZONE_NAME.isin(zone)) | (tmp_sub_labels.matched_zone.isin(zone))]

        # TODO: the fix below is temporary to avoid exception
        if tmp_sub_labels.shape[0] == 0:
            continue
            
        # preprocessing
        if ("ComEd" in zone) and bool(re.match(r"^\d+\s+(\w+)$", name)):
            # COMED's naming convention makes it difficult, and need to be handled separately
            tmp_name = re.search(r"^\d+\s+(\w+)$", name).group(1)        
        elif (("ATSI" in zone) or ("DEOK" in zone) or ("Dayton" in zone)) and bool(re.match(r"^\d+(\w+)$", name)):
             # ATSI, DEOK, and Dayton have naming convention that starts with digits, which make matching difficult
            tmp_name = re.search(r"^\d+(\w+)$", name).group(1)
        else:
            tmp_name = name

        choices = tmp_sub_labels["NAME"].dropna().to_list() # TODO: currently dropping nan, considering replacing with "", i.e. empty string
        
        if index < 3:
            match = process.extractOne(tmp_name, choices)
        else:
            tmp_name = tmp_name.replace("_", "")
            match = find_match(tmp_name, choices)

        if index < 3 and match[1] < threshold:
            continue
        
        if index >= 3 and match[1] > threshold:
            continue
        

        # get substation id and name from substation_labels
        sub_id = tmp_sub_labels[tmp_sub_labels.NAME == match[0]]["SUBSTATION_GLOBALID"].values[0]
        sub_name = tmp_sub_labels[tmp_sub_labels.NAME == match[0]]["NAME"].values[0]

        # add to dictionary
        mapping[name] = (sub_name, zone, sub_id, match[1], index)

        # remove from list and dicionaries
        yet_to_be_matched_sublabels.remove(sub_id)
        del yet_to_be_matched_nodes[name]
    
    # print the iteration number and current number of matching
    print(index, len(mapping.keys()))

0 1723
1 2971
2 3011
3 4084
4 4216
5 4698
6 5427


In [254]:
substation_pnode_match = pd.DataFrame(mapping).T
substation_pnode_match.columns = ["system_map_substation_name", "pnode_zone",
                                  "system_map_substation_id",
                                  "match_score", "match_round"]

In [300]:
# TODO: currently using only the most truthworthy match, which are from the first two rounds

substation_pnode_match[substation_pnode_match.match_round.isin([0, 1, 2])]

,system_map_substation_name,pnode_zone,system_map_substation_id,match_score,match_round
02ANGOLA,Angola,[ATSI],{FD0A8435-45DB-4C01-9D2F-571C762672B1},100,0
02ARMCO,ARMCO,[ATSI],{D5F74F69-1130-4D5F-BC31-95F75F02C577},100,0
02BRUSH,Brush,[ATSI],{424EF662-40EB-46E2-B0C3-9EE5BA8446D1},100,0
02DARWIN,Darwin,[ATSI],{C157ABB7-B65E-4DD0-8FE7-C3C4BBE1A6C9},100,0
02DAWSON,Dawson,[ATSI],{33778DA1-A58B-4EBB-8D0E-BCD20BF30B59},100,0
...,...,...,...,...,...
SWATARA,Swatara,[PPL],{2ED6B6C4-CB5E-40D4-8E67-A9029D7721CF},100,2
THELMAEK,ThelmaEK,[EKPC],{C379C87D-B040-4494-B7EC-E0FFC10E16E7},100,2
USAP,USAP,[DL],{7DE0FF6C-55B8-402F-95AC-8D038A74C996},100,2
USGYPSUM,USGypsum,[AEP],{9B165497-77DF-416A-A7E4-8AFB00B6BA0E},100,2


# Map System Map Lines to Line Rating

In [336]:
# merge line_rating with equiplist
# TODO: currently averaging line rating over different conditions, consider improving this in the future

# load equipment list
equiplist = pd.read_csv("data/oasis/equiplist.csv", skiprows=1)
equiplist["VOLTAGE"] = equiplist["VOLTAGE"].apply(lambda x: float(x.replace("KV", "")))

# merge equiplist with line rating
# TODO: currently disabling line rating
# line_rating = line_rating.groupby(["company", "substation", "voltage", "device", "end", "description"]).mean()
# line_rating = line_rating[["day_normal"]]
# equiplist = pd.merge(equiplist, line_rating, left_on="LONG NAME", right_on="description", how="left")
line_equiplist = equiplist[equiplist.TYPE == "LINE"].copy()

In [260]:
# check if line rating and equiplist are easy to match with each other
# answer is yes
# as seen below, most that are in line rating can be directly matched with equiplist

line_rating = pd.read_csv("data/oasis/line_rating.csv")

for x in line_rating.description.unique():
    if x not in equiplist["LONG NAME"].to_list():
        print(x)

CABINCRE-HERNSHAW CAB-HER
FIELDALE-SHEFFIEL
KAMMER-MUSKINGU 345KV
DELAWAIM-SELMAPAR
ASTOR   -REFUGEE  AST-REF
HILSBORO-MIDD DAY
HYATTCS  1A       XFORMER
HYATTCS  1B       XFORMER
MILTONSW-LICK     MIL-LIC1
NHAVERHI NHAV XF  XFORMER
CORDERCR-PRUNTYTO 12
GORMANIFOLD GN-CE
EMPORADP-SKIPPERS 1029A
TREGO-TREGOTAP 130B1TMP
BLMNTDOM-BRAMBLET 206A
CHOWAN  -SHERTFOR 2131C
CLAYVILL-NWCST_EK CLA-NWCS
SUMSHADT-SUMMER   TIE
INLAND S556_TIE
INLAND S567_TIE
INLAND S578_TIE
INLAND S581_TIE
OYSTERCR NC   CB
OYSTERCR NS   CB
SMITHBUR EB   CB
HOMERCIT 304          CB
HOMERCIT 307          CB
HOMERCIT 308          CB
MERCER   220-1    GSU XFMR
MERCER   220-2    GSU XFMR


In [302]:
# dict(zip(substation_labels["SUBSTATION_GLOBALID"], tuple(zip(substation_labels["NAME"], substation_labels["PLANNING_ZONE_NAME"]))))




In [338]:
# merge lines and substations
lines_tmp = lines[["SUBSTATION_A_GLOBALID", "SUBSTATION_B_GLOBALID", "TO_LINE_NAME", "VOLTAGE"]]
# sub_tmp = substation_labels[["NAME", "SUBSTATION_GLOBALID", "VOLTAGE", "PLANNING_ZONE_NAME"]]
sub_tmp = substation_labels[["NAME", "SUBSTATION_GLOBALID"]]


line_sub = pd.merge(lines_tmp, sub_tmp, how="left",
                    left_on="SUBSTATION_A_GLOBALID", right_on="SUBSTATION_GLOBALID",
                    suffixes=("_LINE", "_SUBSTATION_A"))

line_sub = pd.merge(line_sub, sub_tmp, how="left",
                    left_on="SUBSTATION_B_GLOBALID", right_on="SUBSTATION_GLOBALID",
                    suffixes=("_SUBSTATION_A", "_SUBSTATION_B"))

line_sub = line_sub.drop(columns=["SUBSTATION_GLOBALID_SUBSTATION_A", "SUBSTATION_GLOBALID_SUBSTATION_B"])

# line_sub.LINE_NAME = line_sub.LINE_NAME.apply(lambda x: x.strip() if (x is not np.nan) else x)
# line_sub.LINE_NAME_ALT = line_sub.LINE_NAME_ALT.apply(lambda x: x.strip() if (x is not np.nan) else x)

In [298]:
# # for the lines equipment list, separate and obtain the information on the two connected substations

# # add column for subA, subB
# line_equiplist["subA"] = np.nan
# line_equiplist["subB"] = np.nan

# equiplist_sub_list = list(equiplist.STATION.unique())
# # equiplist_sub_list = list(map(lambda x: re.sub(r'\s+', ' ', x), equiplist_sub_list))

# # find subA and subB for line
# # TODO: currently some substations are missing, come back to fix it by adding more logic to parsing
# for index, row in line_equiplist.iterrows():
#     subA = row["STATION"]
#     longName = row["LONG NAME"]
    
#     longName = re.sub(r'\s+-', '-', longName) # experimenting, eliminating white space before -
    
#     found = False
    
#     # subA-subB somethingsomething
#     for subB in equiplist_sub_list:
#         longName_tmp = "-".join([subA, subB]) + " "
#         if longName_tmp in longName:
#             found = True
#             line_equiplist.loc[index, "subA"] = subA
#             line_equiplist.loc[index, "subB"] = subB
#             break
    
#     # subA-subB
#     if not found:
#         for subB in equiplist_sub_list:
#             longName_tmp = "-".join([subA, subB])
#             if longName_tmp == longName:
#                 found = True
#                 line_equiplist.loc[index, "subA"] = subA
#                 line_equiplist.loc[index, "subB"] = subB
#                 break
    
#     # subB\s or subB
#     if not found:
#         for subB in equiplist_sub_list:
#             if (subB + " ") in longName or bool(re.search(r"{}$".format(subB), longName)):
#                 found = True
#                 line_equiplist.loc[index, "subA"] = subA
#                 line_equiplist.loc[index, "subB"] = subB
#                 break
    
#     # ends in subB\d
#     if not found:
#         for subB in equiplist_sub_list:
#             if bool(re.search(r"{}\d($|\s)".format(subB), longName)):
#                 found = True
#                 line_equiplist.loc[index, "subA"] = subA
#                 line_equiplist.loc[index, "subB"] = subB
#                 break

# line_equiplist.isnull().sum()


In [ ]:
# get substation, company zone mapping information into a single dataframe
station_company_zone_map = pd.concat([equiplist.groupby("STATION")["COMPANY"].unique(),
                                      equiplist.groupby("STATION")["ZONE"].unique()], axis=1)

if station_company_zone_map.COMPANY.apply(lambda x: len(x)).sum() != station_company_zone_map.shape[0]:
    raise

station_company_zone_map.COMPANY = station_company_zone_map.COMPANY.apply(lambda x: x[0])

line_equiplist = pd.merge(line_equiplist,
                          station_company_zone_map.rename(columns={"COMPANY": "subA_COMPANY", "ZONE": "subA_ZONE"}),
                          left_on="subA", right_on="STATION", how="left")

line_equiplist = pd.merge(line_equiplist,
                          station_company_zone_map.rename(columns={"COMPANY": "subB_COMPANY", "ZONE": "subB_ZONE"}),
                          left_on="subB", right_on="STATION", how="left")

# # convert line voltage to float
# line_equiplist.VOLTAGE = line_equiplist.VOLTAGE.apply(lambda x: float(x.replace("KV", "")))
# line_sub.LINE_VOLTAGE = line_sub.LINE_VOLTAGE.astype(float)

In [ ]:
# # match the line equipment list with pjm system map substation names
# # voltage level by voltage level
# line_sub["SUB_A_MATCH"] = line_sub["SUB_A_ID"]
# line_sub["SUB_B_MATCH"] = line_sub["SUB_B_ID"]

# # iterate over voltage
# for voltage_level in [765]: #sorted(lines.VOLTAGE.unique(), reverse=True):    
#     # get substations in system map to be matched
#     tmp = line_sub[line_sub.LINE_VOLTAGE == voltage_level].dropna(subset=["SUB_A_NAME", "SUB_B_NAME"]) # TODO: currently dropping nans, shouldn't be dropping anything in the future
#     sub_match = {**dict(zip(tmp.SUB_A_ID, tmp.SUB_A_NAME)), **dict(zip(tmp.SUB_B_ID, tmp.SUB_B_NAME))}
    
#     # get a list of choices
#     choices = list(line_equiplist[line_equiplist.VOLTAGE == voltage_level].subA.dropna().unique()) # TODO: currently dropping nans, shouldn't be dropping anything in the future
#     choices+= list(line_equiplist[line_equiplist.VOLTAGE == voltage_level].subB.dropna().unique()) # TODO: currently dropping nans, shouldn't be dropping anything in the future
    
#     for s in sub_match.keys():
#         sub_name = sub_match[s]
#         match = process.extractOne(sub_name, choices)
#         sub_match[s] = match[0]
    
#     # TODO: add check based on fuzzy match score
#     # TODO: add check that the matched substations form a line in system map
#     # TODO: add check that the matched substations are from the expected tranmission zone
    
#     # replace
#     line_sub["SUB_A_MATCH"] = line_sub["SUB_A_MATCH"].replace(sub_match)
#     line_sub["SUB_B_MATCH"] = line_sub["SUB_B_MATCH"].replace(sub_match)


# # # deal with left over substations
# # # TODO: this step is extrafluous, should eliminate in the future in favor of a improved step above
# # for index, row in line_sub.iterrows():
# #     if row["SUB_A_MATCH"] == row["SUB_A_ID"]:
# #         line_sub.loc[index, "SUB_A_MATCH"] = np.nan
# #     if row["SUB_B_MATCH"] == row["SUB_B_ID"]:
# #         line_sub.loc[index, "SUB_B_MATCH"] = np.nan

In [306]:
choices = line_equiplist[line_equiplist.VOLTAGE == "765 KV"]["LONG NAME"].to_list()

In [332]:

# initialize dictionaries and lists
mapping = {}


for index, row in line_sub[line_sub.VOLTAGE == "765"].iterrows():
    
    yet_to_be_matched_map_lines = substation_labels.SUBSTATION_GLOBALID.to_list()
    yet_to_be_matched_equiplist_lines = node_list.groupby("substation")["zone"].unique().to_dict()

    
    query = row["NAME_SUBSTATION_A"] + " " + row["NAME_SUBSTATION_B"]
    match = process.extractOne(query, choices, scorer=fuzz.token_sort_ratio)
    
    print(query, "-----", match[0], "-----", match[1])

Maliszewski Marysville ----- MALISZEW-MARYSVI2 MAL-MAR1 ----- 67
Culloden Gavin ----- CULLODE2-GAVINAEP ----- 84
Kammer SouthCanton ----- KAMMER2-SCANTON2 ----- 82
Jefferson Rockport ----- JEFFERSO-ROCKPOR2 ----- 91
Rockport RockportWorks ----- ROCKPOR2-ROCKPWKS2 ----- 80
Cloverdale JoshuaFalls ----- CLOVERD2-JOSHUAF2 ----- 77
Belmont Mountaineer ----- AMOS-MOUNTAIN ----- 69
Greentown Reynolds ----- GREENTO2-REYNOLD2 GRE-REY ----- 70
NorthProctorville HangingRock ----- NPROCTO2-HANGING2 ----- 65
Culloden Baker ----- CULLODE2-BAKER ----- 93
Marysville Flatlick ----- MARYSVI2-FLATLICK ----- 89
Axton JacksonsFerry ----- AXTONAEP-JACKSONS ----- 78
HangingRock DonMarquis ----- CORNU-HANGING ROCK ----- 75
JohnAmos Mountaineer ----- AMOS-MOUNTAIN ----- 79
Dumont Sorenson ----- DUMONT2-SORENSON DUM-SOR ----- 77
Culloden Wyoming ----- CULLODE2-WYOMING2 ----- 91
Maliszewski Vassell ----- MALISZEW-VASSELL  MAL-VAS ----- 79
Dumont WiltonCtr ----- DUMONT2-WILTON ----- 87
CornuHangingRock HangingRoc

In [307]:
choices

['AMOS-CULLODE2',
 'AMOS-MOUNTAIN',
 'AMOS-NPROCTO2',
 'AXTONAEP-JACKSONS',
 'BROADFO2-JACKSONS',
 'CLOVERD2-JACKSONS',
 'CLOVERD2-JOSHUAF2',
 'CULLODE2-BAKER',
 'CULLODE2-GAVINAEP',
 'CULLODE2-WYOMING2',
 'JACKSONS-WYOMING2 JAC-WYO1',
 'KAMMER2-SCANTON2',
 'KAMMER2 -VASSELL  KAM-VAS',
 'MOUNTAIN-GAVINAEP',
 'COOK-DUMONT2',
 'DUMONT2-GREENTO2 TIE',
 'DUMONT2-SORENSON DUM-SOR',
 'DUMONT2-WILTON',
 'JEFFERSO-GREENTO2 TIE',
 'JEFFERSO-ROCKPOR2',
 'ROCKPOR2-ROCKPWKS2',
 'ROCKPOR2-ROCKPWKS3',
 'ROCKPOR2-SULLIVA2',
 'BAKER-BROADFO2',
 'BAKER-HANGING2',
 'CORNU-HANGING ROCK',
 'GAVINAEP-FLATLICK',
 'HANGING2-JEFFERSO',
 'HANGING2-MARQUIS2',
 'MALISZEW-MARYSVI2 MAL-MAR1',
 'MALISZEW-VASSELL  MAL-VAS',
 'MARYSVI2-FLATLICK',
 'MARYSVI2-SORENSON MAR-SOR',
 'NPROCTO2-HANGING2',
 'BELMONT-KAMMER',
 'BELMONT-MOUN AEP',
 'GREENTO2-REYNOLD2 GRE-REY',
 '23 COLLI-112 WILT 11216',
 '23 COLLI-167 PLAN 2315',
 'CHATEAUG-MASSENA',
 'MARCY-MASSENA MSU-1']

In [ ]:
# # replace zonal name to accomodate matching with equiplist
# zonalMap = {'AECO':'AE', 'ATSI':'FE', 'BGE':'BC', 'DAY':'DAYTON', 'DUQ':'DUQU', 'JCPL':'JC',
#             'METED':'ME', 'PECO':'PE', 'PENELEC':'PN', 'PEPCO':'PEP', 'PPL':'PL', 'PSEG':'PS'}

# node_list.zone = node_list.zone.replace(zonalMap)

# # check that all zones are node list can be found in equiplist
# for x in node_list.zone.unique():
#     if x not in equiplist.COMPANY.unique():
#         print(x)
        

# pnode_list_sub_zone_mapping = node_list.groupby("substation")["zone"].unique().apply(lambda x: list(x)).to_dict()
# equiplist_node_pnode_map = {}

# # match pnode list with equipment list
# for query in pnode_list_sub_zone_mapping.keys():
#     sub_zone = pnode_list_sub_zone_mapping[query]
#     choices = list(line_equiplist[line_equiplist.COMPANY.isin(sub_zone)].subA.unique()) #TODO: should it be line_equiplist or equiplist?
#     choices+= list(line_equiplist[line_equiplist.COMPANY.isin(sub_zone)].subB.unique()) #TODO: should it be line_equiplist or equiplist?
#     match = process.extractOne(str(query), choices) #TODO: why do we need str() here?
#     equiplist_node_pnode_map[match[0]] = str(query) #TODO: why do we need str() here?
    
#     # TODO: maybe consider do exact match rather than fuzzy match for better precision
#     # TODO; then use zone as a check
    
#     # TODO: QA/QC on the matching - should be better
#     # TODO: add check based on fuzzy match score
#     # TODO: add check based on voltage
#     # TODO: check matching is one-to-one

In [ ]:
# # map line_sub to pnode

# line_sub["SUB_A_PNODE"] = np.nan
# line_sub["SUB_B_PNODE"] = np.nan

# for index, row in line_sub.iterrows():
#     subAMatch = row["SUB_A_MATCH"]
#     subBMatch = row["SUB_B_MATCH"]
    
#     if subAMatch in equiplist_node_pnode_map.keys():
#         line_sub.loc[index, "SUB_A_PNODE"] = equiplist_node_pnode_map[subAMatch]
#     if subBMatch in equiplist_node_pnode_map.keys():
#         line_sub.loc[index, "SUB_B_PNODE"] = equiplist_node_pnode_map[subBMatch]

# # TODO: need to do QA/QC on this

In [ ]:
# # use contingency list to find missing substations

# # load contingency list
# with open('data/oasis/contingency_list.json') as f:
#     contingency_list = json.load(f)["Contingencies"]["Contingency"]

# contingency_list = {x["Name"]: [y["_b1"] for y in x["Equipment"]["ContingencyEquipment"]] for x in contingency_list}

# for index, row in line_equiplist[line_equiplist.subA.isnull()].iterrows():
#     subA = row["STATION"]
#     longName = row["LONG NAME"]
#     voltage = str(int(row["VOLTAGE"]))
    
#     matchedGroups = re.search(r"^([\w\s.]+)-\s?([\w.]+)", longName)
    
#     subB = matchedGroups.group(2)

# #     print(longName)
#     for contingency in contingency_list.keys():
#         tmp = contingency.lower()
#         if subB.lower() in tmp:
#             match = process.extractOne(subB, contingency_list[contingency])
#             print(longName, "---", subB, match[0], match[1])
#             break

In [ ]:
# if 500 kv, skip

In [ ]:
# QA/QC
# are all lines in equiplist present in pjm system map?
# are all lines in pjm system map present in equiplist?

# after mapping, manual review those match with low fuzzy match/levenshtein score

# use existing zipcode mapping to do QA/QC

# Load Generation (Planning Queue & EIA)

In [ ]:
# load planning queue data

# load data
with open('data/pjm_system_map_export/queue.json') as f:
    data = json.load(f)
print("number of data points:", len(data["results"]))


# check data
for x in data["results"]:
    if x["layerId"] != 0:
        raise("Failed layerId check!")
    if x["layerName"] != "Queues":
        raise("Failed layerName check!")
    if x["geometryType"] != "esriGeometryPoint":
        raise("Failed geometryType check!")
print("passed data check")


# construct geojson
geojson = {
    "type": "FeatureCollection",
    "crs": { "type": "name", "properties": { "name": "urn:ogc:def:crs:OGC:1.3:CRS84" } }, # this is not the appropriate projection
    "features": [
    {
        "type": "Feature",
        "properties" : d["attributes"],
        "geometry" : {
            "type": "Point",
            "coordinates": [d["geometry"]["x"], d["geometry"]["y"]],
            }
     } for d in data["results"]]
}

print("finished constructing geojson")


# write geojson to disk
output = open("data/pjm_system_map_export/queue.geojson", 'w')
json.dump(geojson, output)
output.close()
print("geojson file has been written out to disk!")

# load geojson as a geodataframe
queue = gpd.read_file("data/pjm_system_map_export/queue.geojson")
print("geojson file has been read as a geodataframe!")

# replace NULL and None values with NaN
queue = queue.replace({"Null": np.nan, None: np.nan, "":np.nan})

# load queue information exported from PJM as excel
queue_info = pd.read_excel("data/PlanningQueues_05_25_2020.xlsx")
queue_merged = pd.merge(queue, queue_info, how="left", left_on="QUEUE_ID", right_on="Queue Number")
columns = ["QUEUE_ID", "Name", "MFO", "MW Energy", "MW Capacity", "VOLTAGE",
           "MW In Service", "Project Type", "Fuel", "Status", "Revised In Service Date",
           "Actual In Service Date", "geometry"]
queue_merged = queue_merged[columns]
queue_merged = queue_merged[queue_merged["Status"] == "In Service"]
print("finished merging queue geojson and queue planning info")
print("number of data points:", queue_merged.shape[0])

In [ ]:
queue_merged.geometry.plot(figsize=(25, 5))

In [ ]:
queue_merged.head()

In [ ]:
# using EIA 860 to map generators to node and coordinates
# currently using the 2018 data

# load data
eia_860_plant = pd.read_excel("data/eia8602018/2___Plant_Y2018.xlsx", skiprows=1)
eia_860_gens = pd.read_excel("data/eia8602018/3_1_Generator_Y2018.xlsx", skiprows=1).iloc[:-1, :]
eia_860_plant["Plant Code"] = eia_860_plant["Plant Code"].astype(int)
eia_860_gens["Plant Code"] = eia_860_gens["Plant Code"].astype(int)


# # select relevant columns
eia_860_gens = eia_860_gens[["Plant Code", "Generator ID", "Unit Code", "Technology", "Prime Mover",
                             "RTO/ISO LMP Node Designation",
                             "RTO/ISO Location Designation for Reporting Wholesale Sales Data to FERC"]]

eia_860_plant = eia_860_plant[["Plant Code", "Plant Name", "Street Address", "City", "County", "State",
                               "Latitude", "Longitude", "Balancing Authority Name",
                               "Transmission or Distribution System Owner"]]

# check overlapping values
# how many plants that are not in gens?
plant_code_not_found = []
for x in eia_860_plant["Plant Code"].unique():
    if x not in eia_860_gens["Plant Code"].unique():
        plant_code_not_found.append(x)
print("in '2___Plant' but no in '3_1_Generator':", len(plant_code_not_found))

# how many in gens that are not in plant?
plant_code_not_found = []
for x in eia_860_gens["Plant Code"].unique():
    if x not in eia_860_plant["Plant Code"].unique():
        plant_code_not_found.append(x)
print("in '3_1_Generator' but not in '2___Plant':", len(plant_code_not_found))

# merge dataframe and select PJM
plants = pd.merge(eia_860_plant, eia_860_gens, on="Plant Code", how="outer")
plants["Plant Code"] = plants["Plant Code"].astype(str)
# plants = plants[plants["Balancing Authority Name"] == "PJM Interconnection, LLC"]

# eliminate rows that do not have coordinates
# TODO: figure out if there are other ways to get coordinates
plants = plants[(plants.Longitude != " ") & (plants.Latitude != " ")]

# construct geodataframe
plants = gpd.GeoDataFrame(plants, geometry=gpd.points_from_xy(plants.Longitude, plants.Latitude))

# convert projection
plants.crs = 4326
plants = plants.to_crs(epsg = 3857)

In [ ]:
plants

In [ ]:
# for index, row in queue_merged.iterrows():
#     queue_id = row["QUEUE_ID"]
#     queue_geo = row["geometry"]
    
#     min_distance = plants.geometry.apply(lambda x: x.distance(queue_geo)).min()
    
#     if min_distance < 1000:
#         print(queue_id, min_distance)

In [ ]:
from shapely.ops import nearest_points

# unary union of the gpd2 geomtries 
gpd1 = plants
gpd2 = queue_merged

pts3 = gpd2.geometry.unary_union
def near(point, pts=pts3):
    nearest = gpd2.geometry == nearest_points(point, pts)[1]
    return gpd2[nearest].QUEUE_ID.values[0]

gpd1['Nearest'] = None

for index, row in gpd1.iterrows():
    try:
        gpd1.loc[index, 'Nearest'] = near(row["geometry"])
    except:
        print("Problemaitc geometry:", index, row["Plant Code"])

# gpd1.apply(lambda row: near(row.geometry), axis=1)
# gpd1.apply(lambda row: near(row.geometry), axis=1)

In [ ]:
tmp = pd.merge(gpd1, gpd2, left_on="Nearest", right_on="QUEUE_ID", how="left")

In [ ]:
tmp.columns

In [ ]:
tmp[tmp["Balancing Authority Name"] == "PJM Interconnection, LLC"].to_csv('tmp.csv')

In [ ]:
# # get substation in lines data

# ax = states.plot(figsize=(30, 10))
# pjm_zone.geometry.plot(ax=ax, color="green")
# lines.geometry.plot(ax=ax, color="yellow")
# substations.geometry.plot(ax=ax, color="red")
# # substations[~substations.PLANNING_ZONE_NAME.isnull()].geometry.plot(ax=ax, color="red")

# plants[plants["Balancing Authority Name"] == "PJM Interconnection, LLC"].geometry.plot(ax=ax, color="black")

# queue_merged.geometry.plot(ax=ax, color="white")

In [ ]:
# plants[plants["Balancing Authority Name"] == "PJM Interconnection, LLC"]["Plant Code"]

# Load Attachment

In [ ]:
pjm_zone.geometry.apply(lambda x: x.is_valid)

#TODO: APS is still false for some reason. May be because it's a combination of donut and multipolygon shape?

In [ ]:
# convert state boundaries

# load state
with open('data/pjm_system_map_export/states.json') as f:
    data = json.load(f)
    
print("number of states:", len(data["results"]))


# check data
for x in data["results"]:
    if x["layerId"] != 18:
        raise("Failed layerId check!")
    if x["layerName"] != "States":
        raise("Failed layerName check!")
    if x["geometryType"] != "esriGeometryPolygon":
        raise("Failed geometryType check!")


# construct geojson
geojson = {
    "type": "FeatureCollection",
    "crs": { "type": "name", "properties": { "name": "urn:ogc:def:crs:OGC:1.3:CRS84" } },
    "features": []
}


# add to geojson
for d in data["results"]:
    tmp = {"type": "Feature", "properties" : d["attributes"], "geometry":{"type": None, "coordinates": None}}
    
    if len(d["geometry"]["rings"]) == 1:
        tmp["geometry"]["type"] = "Polygon"
        geojson["features"].append(tmp)
        tmp["geometry"]["coordinates"] = d["geometry"]["rings"]
    else:
        tmp["geometry"]["type"] = "MultiPolygon"
        geojson["features"].append(tmp)
        tmp["geometry"]["coordinates"] = [[x] for x in d["geometry"]["rings"]]
    

# write geojson to disk
output = open("data/pjm_system_map_export/states.geojson", 'w')
json.dump(geojson, output)
output.close()
print("geojson file has been written out to disk!")

# load geojson as a geodataframe
states = gpd.read_file("data/pjm_system_map_export/states.geojson")
print("geojson file has been read as a geodataframe!")

# replace NULL and None values with NaN
states = states.replace({"Null": np.nan, None: np.nan, "":np.nan})

# select states that are in PJM states
states = states[states.ABBREVIATION.isin(["DE", "IL", "IN", "KY", "MD", "MI", "NJ", "NC", "OH", "PA", "TN", "VA", "WV", "DC"])]

In [ ]:
# get substation in lines data
substations = substations[substations.SUBSTATION_GLOBALID.isin(lines.SUBSTATION_A_GLOBALID) | substations.SUBSTATION_GLOBALID.isin(lines.SUBSTATION_B_GLOBALID)]

ax = states.plot(figsize=(30, 10))
pjm_zone.geometry.plot(ax=ax, color="green")
lines.geometry.plot(ax=ax, color="yellow")
# substations.geometry.plot(ax=ax, color="red")
substations[~substations.PLANNING_ZONE_NAME.isnull()].geometry.plot(ax=ax, color="red")

In [ ]:
# load data

load = pd.read_csv("data/hrl_load_metered_2015.csv")
load.datetime_beginning_utc = pd.to_datetime(load.datetime_beginning_utc)
load.datetime_beginning_ept = pd.to_datetime(load.datetime_beginning_ept)

In [ ]:
load.head(2)

In [ ]:
# method 1, use tranmission zone
# TODO: in the future, use load area, which requires further mapping based on EDC service areas
# TODO: in the future, use voronoi cell for better approximation

# get matching for load zone matching
mapping = {
    'Eastern Kentucky Power Cooperative': 'EKPC',
    'Duke Energy Ohio Kentucky': 'DEOK',
    'Commonwealth Edison Company': 'CE',
    'American Electric Power Co., Inc.': 'AEP',
    'American Transmission Systems, Inc.': 'ATSI',
    'Allegheny Power': 'AP',
    'Duquesne Light Company': 'DUQ',
    'Public Service Electric and Gas Company': 'PS',
    'Pennsylvania Electric Company': 'PN',
    'PECO Energy Company': 'PE',
    'Virginia Electric and Power Co.': 'DOM',
    'PPL Electric Utilities Corporation': 'PL',
    'Potomac Electric Power Company': 'PEP',
    'The Dayton Power and Light Co.': 'DAY',
    'Delmarva Power and Light Company': 'DPL',
    'Jersey Central Power and Light Company': 'JC',
    'Rockland Electric Company': 'RECO',
    'Metropolitan Edison Company': 'ME',
    'Baltimore Gas and Electric Company': 'BC',
    'Atlantic City Electric Company': 'AE',
    'Ohio Valley Electric Corporation': 'OVEC'
}

# check mapping
# zones = ['PS', 'PE', 'PL', 'BC', 'JC', 'ME', 'PN', 'PEP', 'AE',
#          'DPL', 'RECO', 'AP', 'CE', 'AEP', 'DAY', 'DUQ', 'DEOK',
#          'ATSI', 'EKPC', 'DOM']

for x in load.zone.unique():
    if x not in mapping.values():
        print(x)
        
print()

for x in mapping.values():
    if x not in load.zone.unique():
        print(x)


# replace commercial zones in substations
substations.COMMERCIAL_ZONE = substations.COMMERCIAL_ZONE.replace(mapping)

In [ ]:
# construct substation loads dataframe
# substations = substations[~substations.COMMERCIAL_ZONE.isin(["Null", "OVEC"])]

load_dict = {x:{} for x in substations.SUBSTATION_KEY}

# figure out share of the load for each substation
# TODO: currently equally divided between substations in a zone. Use Voronoi cells in the future.
substations["loadAreaShare"] = None

for z in substations.COMMERCIAL_ZONE.unique():
    tmp = substations[substations.COMMERCIAL_ZONE == z]
    num_sub = tmp.shape[0]
    substations.loc[tmp.index, "loadAreaShare"] = 1/num_sub

In [ ]:
# construct substation loads dataframe
for index, row in substations.iterrows():
    subKey = row["SUBSTATION_KEY"]
    loadAreaShare = row["loadAreaShare"]
    zone = row["COMMERCIAL_ZONE"]
    
    # if commercial zone is OVEC, skip
    # TODO
    if zone == "OVEC" or zone == "Null":
        continue
    
    estimatedLoad = load[load.zone != "RTO"].groupby(["zone", "datetime_beginning_utc"]).sum().loc[zone] * loadAreaShare
    load_dict[subKey] = estimatedLoad.to_dict()['mw']

# construct DataFrame
pypsa_load = pd.DataFrame(load_dict)

# fillna
pypsa_load = pypsa_load.fillna(0)

In [ ]:
# # load attachment method 2
# # TODO: voronoi cells


# # get dict of zonal information in the form of {zone: geometry}
# zonal_area = {row["PLANNING_ZONE_NAME"]:row["geometry"] for _, row in pjm_zone.iterrows()}

# # reindex substations
# substations.reset_index(drop=True, inplace=True)

# # add a column to store within search info
# substations["geometryZone"] = None

# # check zone
# for index, row in substations.iterrows():
#     point = row["geometry"]
#     for zone, shape in zonal_area.items():
#         if point.within(shape):
#             substations.loc[index, "geometryZone"] = zone
#             break

# # drop those without a geometryZone
# substations = substations.dropna(subset=["geometryZone"], axis=0)

In [ ]:
# # intialize list to store voronoi area shape
# voronoiGeometry = [None for _ in range(substations.shape[0])]
# voronoiArea = [None for _ in range(substations.shape[0])]

# for z in pjm_zone.PLANNING_ZONE_NAME.unique():
#     # get list of substations
#     tmp = substations[substations.geometryZone == z]
    
#     # if shape 0, skip (OVEC)
#     # TODO: need to deal with this later
#     if tmp.shape[0] == 0:
#         continue
    
#     # get shape of the zone
#     area_shape = zonal_area[z]
    
#     # get list of substation points
#     pts = tmp.geometry.to_list()
    
#     # convert to coordinates
#     coords = points_to_coords(pts)
    
#     # calculate Voronoi cells
#     poly_shapes, pts, poly_to_pt_assignments = voronoi_regions_from_coords(coords, area_shape)
#     poly_areas = calculate_polygon_areas(poly_shapes, m2_to_km2=True)
    
#     # store polyshape associated with a point in the dataframe
#     for i, value in enumerate(poly_to_pt_assignments):
#         index = tmp.index[value][0]
#         voronoiGeometry[index] = poly_shapes[i]
#         voronoiArea[index] = poly_areas[i]

# # add Voronoi area to the dataframe
# substations["voronoiGeometry"] = gpd.GeoSeries(voronoiGeometry)
# substations["voronoiArea"] = pd.Series(voronoiArea)

In [ ]:
# QA/QC
# TODO: need to check voronoiArea and voronoiGeometry are correctly in place

# Reformat DataFrames for PyPSA

In [ ]:
# define projection

project = partial(
    pyproj.transform,
    pyproj.Proj('epsg:3857'), # source coordinate system
    pyproj.Proj('epsg:4326')) # destination coordinate system

In [ ]:
# format substation data for PyPSA as buses

# load dataframe, change and add some columns
# TODO: currently simply dropping all duplicates
pypsa_sub = substations.drop_duplicates().copy(deep=True)
pypsa_sub["frequency"] = 60
pypsa_sub["type"] = "substation"

# convert spatial reference to coordinates
pypsa_sub.geometry = pypsa_sub.geometry.apply(lambda x: transform(project, x))

# format dataframe for outputing
pypsa_sub["x"] = pypsa_sub.geometry.y
pypsa_sub["y"] = pypsa_sub.geometry.x #pypsa_sub.y.apply(lambda x: '{:.4f}'.format(x))
pypsa_sub["wkt_srid"] = "SRID=4326;POINT(" + pypsa_sub.x.astype(str) + " " + pypsa_sub.y.astype(str) + ")"
pypsa_sub = pypsa_sub[["SUBSTATION_KEY", "frequency", "COMMERCIAL_ZONE", "NAME", "type", "VOLTAGE", "wkt_srid", "x", "y"]]
pypsa_sub.columns = ["name", "frequency", "operator", "osm_name", "type", "voltage", "wkt_srid", "x", "y"]

In [ ]:
# format lines

# get SUBSTATION_GLOBALID and SUBSTATION_KEY key_value pair
sub_dict = substations[["SUBSTATION_GLOBALID", "SUBSTATION_KEY", ]].drop_duplicates().set_index("SUBSTATION_GLOBALID").to_dict(orient="index")

# initialize key
pypsa_lines = {"name":[], "bus0":[], "bus1":[], "frequency":[], "length":[], "operator":[],
               "osm_name":[], "voltage":[]
              }

# iterate over rows
for index, row in lines.iterrows():
    try:
        name = row["TRANSMISSION_LINE_KEY"]
        bus0 = sub_dict[row["SUBSTATION_A_GLOBALID"]]["SUBSTATION_KEY"]
        bus1 = sub_dict[row["SUBSTATION_B_GLOBALID"]]["SUBSTATION_KEY"]
        frequency = 60
        length = row["LENGTH_KM"]
        operator = row["NAME"]
        osm_name = row["LINE_ID"]
        voltage = row["VOLTAGE"]

        pypsa_lines["name"].append(name)
        pypsa_lines["bus0"].append(bus0)
        pypsa_lines["bus1"].append(bus1)
        pypsa_lines["frequency"].append(frequency)
        pypsa_lines["length"].append(length)
        pypsa_lines["operator"].append(operator)
        pypsa_lines["osm_name"].append(osm_name)
        pypsa_lines["voltage"].append(voltage)
    except:
        print("can't append line: TRANSMISSION_LINE_KEY", row["TRANSMISSION_LINE_KEY"])

# initialize dataframe from dict
pypsa_lines = pd.DataFrame(pypsa_lines)

# drop lines where length is null
# TODO: find some other way in the future
# TODO: perhaps apporximate using path geometry or euclidean distance between substations
pypsa_lines = pypsa_lines[pypsa_lines.length != "Null"]

In [ ]:
# format generators for PyPSA

# copy queue_merged into a geneartor dataframe
pypsa_gens = queue_merged[["Name", "MW In Service", "Fuel", "geometry"]].copy(deep=True)
pypsa_gens["substation"] = ""

# get subs that show up in lines
lst_of_subs = substations[substations.SUBSTATION_KEY.isin(pypsa_lines.bus0) | substations.SUBSTATION_KEY.isin(pypsa_lines.bus1)].copy(deep=True)
lst_of_subs["tmp_dist"] = None

# associate generators to closet subs
for index, row in pypsa_gens.iterrows():
    pt = row["geometry"]
    lst_of_subs["tmp_dist"] = lst_of_subs.geometry.apply(lambda x: pt.distance(x))
    
    min_sub = lst_of_subs[lst_of_subs.tmp_dist == min(lst_of_subs.tmp_dist)]
    min_sub = min_sub.SUBSTATION_KEY.to_list()[0]
    
    pypsa_gens.loc[index, "substation"] = min_sub

# reformat columns for pypsa
pypsa_gens = pypsa_gens[["Name", "substation", "MW In Service", "Fuel"]]
pypsa_gens.columns = ["name", "bus", "p_nom", "carrier"]

# get marginal cost
pypsa_gens["marginal_cost"] = 30

# drop null rows
# TODO: better handling in the future
pypsa_gens.dropna(inplace=True)

# set p_max_pu
# TODO: set the actual p_max_pu for renewable
# pypsa_gens["p_max_pu"] = 1

# deal with duplicates
# TODO: figure out why they are duplicates in the future
pypsa_gens.drop_duplicates(subset="name", inplace=True)

In [ ]:
queue_merged[queue_merged.Name == "Arnold 115kV"]

In [ ]:
substations[substations.SUBSTATION_KEY == "79429"]

In [ ]:
pypsa_gens

In [ ]:
# TODO
# QA/QC on gen-substation approximation

In [ ]:
pypsa_sub

In [ ]:
pypsa_sub

In [ ]:
pypsa_load

In [ ]:
pypsa_gens.groupby("carrier").sum()

In [ ]:
tmp = pd.read_csv("data/gen_by_fuel_2019.csv")
tmp.datetime_beginning_utc = pd.to_datetime(tmp.datetime_beginning_utc)
tmp = tmp[["datetime_beginning_utc", "fuel_type", "mw"]]

tmp.pivot(index="datetime_beginning_utc", columns="fuel_type", values="mw")

# Export Data

In [ ]:
# export csv

pypsa_dir = "data/pjm_system_map_pypsa"

# export dataframes
pypsa_sub.to_csv(os.path.join(pypsa_dir, "buses.csv"), index=False)
pypsa_lines.to_csv(os.path.join(pypsa_dir, "lines.csv"), index=False)
pypsa_gens.to_csv(os.path.join(pypsa_dir, "generators.csv"), index=False)
pypsa_load.to_csv(os.path.join(pypsa_dir, "loads-p_set.csv"))

# generate load name matching
tmp = pd.DataFrame(columns=["name", "bus"])
tmp["name"] = pypsa_sub["name"]
tmp["bus"] = pypsa_sub["name"]
tmp.to_csv(os.path.join(pypsa_dir, "loads.csv"), index=False)


# generate snapshots
tmp = pd.DataFrame(columns=["name", "weights"])
tmp["name"] = pypsa_load.index
tmp["weights"] = 1
tmp.to_csv(os.path.join(pypsa_dir, "snapshots.csv"), index=False)

# Start PyPSA

In [ ]:
# get PyPSA Network

csv_folder_name = "data/pjm_system_map_pypsa"

network = pypsa.Network(csv_folder_name=csv_folder_name)

In [ ]:
fig,ax = plt.subplots(1,1,subplot_kw={"projection":ccrs.PlateCarree()})

fig.set_size_inches(10,10)

load_distribution = network.loads_t.p_set.loc[network.snapshots[0]].groupby(network.loads.bus).sum()

network.plot(bus_sizes=0.00005*load_distribution,ax=ax,title="Load distribution")

In [ ]:
pypsa_gens.carrier.unique()

In [ ]:
fig,ax = plt.subplots(1,1,subplot_kw={"projection":ccrs.PlateCarree()})
fig.set_size_inches(10,10)

gens = network.generators[network.generators.carrier == "Natural Gas"]
gen_distribution = gens.groupby("bus").sum()["p_nom"].reindex(network.buses.index,fill_value=0.)

network.plot(bus_sizes=0.0001*gen_distribution,ax=ax,title="Generation distribution")

In [ ]:
network.lines.s_nom = 1000

In [ ]:
# import export
# generator capacities

In [ ]:
network.generators.p_nom.sum()

In [ ]:
network.loads_t["p_set"].sum(axis=1)

In [ ]:
network.lines.s_nom

In [ ]:
group_size = 4

solver_name = "glpk"

print("Performing linear OPF for one day, {} snapshots at a time:".format(group_size))

network.storage_units.state_of_charge_initial = 0.

for i in range(int(24/group_size)):
    #set the initial state of charge based on previous round
    network.lopf(network.snapshots[group_size*i:group_size*i+group_size], solver_name=solver_name)
    